In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import re

# --- 1. Chargement des Données ---
# Ajustez le chemin si nécessaire
try:
    df = pd.read_csv('C:/Users/Infinix/Desktop/Projet fil rouge/data/processed/medical_dataset_ml.csv')
except FileNotFoundError:
    print("Erreur: Fichier 'data/processed/medical_dataset_ml.csv' non trouvé.")
    # Quittez ou gérez l'erreur comme vous le souhaitez
    exit()

print(f"Données chargées : {df.shape[0]} lignes.")

# --- 2. Nettoyage simple (pour TF-IDF) ---
# Le TF-IDF gère les virgules comme séparateurs si on les remplace par des espaces
def clean_symptoms(text):
    if not isinstance(text, str):
        return ""
    # Remplace les underscores et virgules par des espaces
    text = re.sub(r'[_,]', ' ', text)
    # Supprime les espaces superflus
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['symptoms_processed'] = df['All_Symptoms'].apply(clean_symptoms)

# --- 3. Définition des Features (X) et de la Cible (y) ---
FEATURES = ['symptoms_processed', 'Symptom_Count', 'Severity_Score']
TARGET = 'Disease'

X = df[FEATURES]
y = df[TARGET]

# --- 4. Encodage de la Cible (Maladies) ---
# XGBoost a besoin d'étiquettes numériques (0, 1, 2...)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
# Gardons les noms des classes pour le rapport final
class_names = le.classes_

# --- 5. Séparation des Données (Train / Test) ---
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# --- 6. Pipeline de Prétraitement (Le Cœur) ---

# Définir quel transformeur s'applique à quelle colonne
preprocessor = ColumnTransformer(
    transformers=[
        # Appliquer TF-IDF à la colonne 'symptoms_processed'
        ('tfidf', TfidfVectorizer(max_features=150), 'symptoms_processed'),
        
        # Appliquer un StandardScaler aux colonnes numériques
        ('numeric', StandardScaler(), ['Symptom_Count', 'Severity_Score'])
    ],
    remainder='passthrough' # Ne rien faire aux autres colonnes (s'il y en avait)
)

# --- 7. Création du Pipeline Complet (Préprocesseur + Modèle) ---

# On utilise le classificateur XGBoost
# Note : XGBoost gère nativement les étiquettes encodées de 0 à N-1
model = xgb.XGBClassifier(
    use_label_encoder=False,  # Recommandé dans les versions récentes
    eval_metric='mlogloss',   # Métrique d'évaluation pour la classification multi-classes
    random_state=42
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# --- 8. Entraînement ---
print("\nEntraînement du modèle XGBoost avec TF-IDF...")
pipeline.fit(X_train, y_train)
print("Entraînement terminé.")

# --- 9. Évaluation ---
print("\n--- Évaluation sur l'ensemble de test ---")
y_pred = pipeline.predict(X_test)

# Affichage des métriques
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Précision (Accuracy) : {accuracy * 100:.2f} %")

print("\n--- Rapport de Classification Détaillé ---")
# Convertir les prédictions numériques en noms de maladies
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)

print(classification_report(y_test_labels, y_pred_labels, zero_division=0))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import re